<a href="https://colab.research.google.com/github/jnsbrdbr/thesis/blob/thesis/TUNbalance_Gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install TensorFlow
# !pip install -q tensorflow-gpu==2.0.0-rc0

try:
  %tensorflow_version 2.x  # Colab only.
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # Colab only.`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.
2.8.0


In [2]:
import keras
from keras.layers import Input, Dense, Activation, LeakyReLU, BatchNormalization
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
import keras.backend as K
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip3 install pickle5
import pickle5 as pickle

In [5]:
with pd.HDFStore('/content/drive/MyDrive/1000/TUnsmote_result',  mode='r') as newstore:
    result = newstore.select('result')

In [6]:
#final data
print(result)

            0         1         2  ...             8             9  label
0   -1.715972 -3.099936 -0.041249  ... -1.071834e-02 -9.464034e-03      1
1   -1.715972 -3.099936 -0.041249  ... -3.168693e-02 -7.296139e-03      1
2   -1.715972 -3.099936 -0.041249  ...  7.887649e-03 -4.402416e-02      1
3   -1.715972 -3.099936 -0.041249  ...  8.369804e-03  2.997764e-02      1
4   -1.715972 -3.099936 -0.041249  ...  3.817151e-13  4.136409e-16      1
..        ...       ...       ...  ...           ...           ...    ...
995 -2.114893 -4.572474 -0.754679  ...  2.281326e-03  5.690586e-03      1
996 -2.114893 -4.572474 -0.754679  ... -3.257351e-03 -1.247434e-02      1
997 -2.114893 -4.572474 -0.754679  ... -1.938248e-02 -7.635838e-03      1
998 -2.114893 -4.572474 -0.754679  ... -2.508770e-02 -7.523238e-03      1
999 -2.114893 -4.572474 -0.754679  ... -1.098104e-02  1.125916e-03      1

[1000 rows x 71 columns]


In [7]:
#divide data in to data_label and non_data label

import copy
result_nonlabel=copy.deepcopy(result)
del result_nonlabel['label']
X=result_nonlabel
y=result.label

In [8]:
#splitting data in to test and train...

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y ,random_state=42,test_size=0.3)#

In [9]:
print('shape of x_train is:',X_train.shape)
print('shape of x_test is:',X_test.shape)

shape of x_train is: (700, 70)
shape of x_test is: (300, 70)


In [10]:
def make_latent_samples(n_samples, sample_size):
    #return np.random.uniform(-1, 1, size=(n_samples, sample_size))
    return np.random.normal(loc=0, scale=1, size=(n_samples, sample_size))

In [11]:
make_latent_samples(1, 300) # generates one sample

array([[-1.49823750e+00,  7.22429572e-01, -3.60917986e-01,
        -2.91323317e-01,  5.59640133e-01, -5.14243056e-01,
         6.49092721e-01, -1.89695217e+00,  7.23751164e-01,
         5.67027468e-01, -1.08555078e+00, -5.97402363e-01,
         6.01369786e-01,  5.18162778e-01,  1.76531447e+00,
         1.05491528e-01,  1.31613117e+00,  2.16851655e-01,
        -1.66942271e-01, -1.14968739e+00, -1.05574779e+00,
         1.61189049e+00, -1.44760281e-01,  1.12197860e+00,
        -2.35651674e-02, -7.49966372e-01, -3.72494159e-01,
         1.55878609e-01, -6.22258993e-01,  1.24739106e+00,
        -1.74005816e-01,  4.54142619e-01, -2.85171655e-01,
         6.94516447e-01,  8.60891940e-01,  3.96029973e-01,
         2.70072193e-01,  5.79524425e-02, -3.13822411e-01,
        -1.11599597e+00,  5.12396319e-01, -6.74550187e-01,
        -5.65188321e-01,  1.06518530e+00,  1.82161644e+00,
         1.05832095e+00, -1.85052990e+00,  8.57928104e-01,
         4.40248856e-01, -4.27466064e-01,  9.28430262e-0

In [12]:
generator = Sequential([
    Dense(256, input_shape=(100,)),
    LeakyReLU(alpha=0.02),
    #BatchNormalization(momentum=0.6),
    #Dense(512),
    #LeakyReLU(alpha=0.02),
    #BatchNormalization(momentum=0.6),
    #Dense(256),
    #LeakyReLU(alpha=0.02),
    #BatchNormalization(momentum=0.6),
    #Dense(128),
    #LeakyReLU(alpha=0.02),
    #BatchNormalization(momentum=0.6),
    Dense(70),
    Activation('tanh')
], name='generator')

generator.summary()

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               25856     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 70)                17990     
                                                                 
 activation (Activation)     (None, 70)                0         
                                                                 
Total params: 43,846
Trainable params: 43,846
Non-trainable params: 0
_________________________________________________________________


In [13]:
discriminator = Sequential([
    Dense(128, input_shape=(70,)),
    LeakyReLU(alpha=0.02),
    #(momentum=0.6),
    Dense(64),
    LeakyReLU(alpha=0.02),
    #BatchNormalization(momentum=0.6),
    Dense(32),
    LeakyReLU(alpha=0.02),
    #BatchNormalization(momentum=0.6),
    #Dense(16),
    #LeakyReLU(alpha=0.02),
    #BatchNormalization(momentum=0.6),
    #Dense(8),
    #LeakyReLU(alpha=0.02),
    #BatchNormalization(momentum=0.6),
    Dense(1),
    Activation('sigmoid')
], name='discriminator')

discriminator.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 128)               9088      
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 32)                0         
                                                                 
 dense_5 (Dense)             (None, 1)               

In [14]:
# maintain the same shared weights with the generator and the discriminator.
gan = Sequential([
    generator,
    discriminator
])

gan.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 generator (Sequential)      (None, 70)                43846     
                                                                 
 discriminator (Sequential)  (None, 1)                 19457     
                                                                 
Total params: 63,303
Trainable params: 63,303
Non-trainable params: 0
_________________________________________________________________


In [15]:
def make_trainable(model, trainable):
    for layer in model.layers:
        layer.trainable = trainable

In [16]:
make_trainable(discriminator, False)
discriminator.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 128)               9088      
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 32)                0         
                                                                 
 dense_5 (Dense)             (None, 1)               

In [17]:
make_trainable(discriminator, True)
discriminator.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 128)               9088      
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 32)                0         
                                                                 
 dense_5 (Dense)             (None, 1)               

In [18]:
def make_simple_GAN(sample_size, 
                    g_hidden_size_1,
                    g_hidden_size_2,
                    g_hidden_size_3,
                    g_hidden_size_4,
                    d_hidden_size_1,
                    d_hidden_size_2, 
                    d_hidden_size_3,
                    d_hidden_size_4,
                    d_hidden_size_5,
                    leaky_alpha, 
                    g_learning_rate,
                    d_learning_rate):
    K.clear_session()
    
    generator = Sequential([
        Dense(g_hidden_size_1, input_shape=(sample_size,)),
        LeakyReLU(alpha=leaky_alpha),
        #BatchNormalization(momentum=0.6),
        #Dense(g_hidden_size_2),
        #LeakyReLU(alpha=leaky_alpha),
        #BatchNormalization(momentum=0.6),
        #Dense(g_hidden_size_3),
        #LeakyReLU(alpha=leaky_alpha),
        #BatchNormalization(momentum=0.6),
        #Dense(g_hidden_size_4),
        #LeakyReLU(alpha=leaky_alpha),
        #BatchNormalization(momentum=0.6),
        Dense(70),        
        Activation('tanh')
    ], name='generator')    

    discriminator = Sequential([
        Dense(d_hidden_size_1, input_shape=(70,)),
        LeakyReLU(alpha=leaky_alpha),
        #BatchNormalization(momentum=0.6),
        Dense(d_hidden_size_2),
        LeakyReLU(alpha=leaky_alpha),
        #BatchNormalization(momentum=0.6),
        Dense(d_hidden_size_3),
        LeakyReLU(alpha=leaky_alpha),
        #BatchNormalization(momentum=0.6),
        #Dense(d_hidden_size_4),
        #LeakyReLU(alpha=leaky_alpha),
        #BatchNormalization(momentum=0.6),
        #Dense(d_hidden_size_5),
        #LeakyReLU(alpha=leaky_alpha),
        #BatchNormalization(momentum=0.6),
        Dense(1),
        Activation('sigmoid')
    ], name='discriminator')    
    
    gan = Sequential([
        generator,
        discriminator
    ])
    
    discriminator.compile(optimizer=Adam(lr=d_learning_rate), loss='binary_crossentropy')
    gan.compile(optimizer=Adam(lr=g_learning_rate), loss='binary_crossentropy')
    
    return gan, generator, discriminator 

In [19]:
from sklearn import preprocessing
def preprocess(x):
   x = preprocessing.scale(x)
   return x

In [20]:
X_train_real = np.array(preprocess(X_train))
X_test_real  = np.array(preprocess(X_test))

In [21]:
def make_labels(size):
    return np.ones([size, 1]), np.zeros([size, 1])

In [22]:
y_real_10, y_fake_10 = make_labels(10)

y_real_10, y_fake_10

(array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]]), array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]))

In [23]:
# hyperparameters
sample_size     = 100     # latent sample size (i.e., 100 random numbers)
g_hidden_size_1 = 256
g_hidden_size_2 = 0
g_hidden_size_3 = 0
g_hidden_size_4 = 0
d_hidden_size_1 = 128
d_hidden_size_2 = 64
d_hidden_size_3 = 32
d_hidden_size_4 = 16
d_hidden_size_5 = 8
leaky_alpha     = 0.02
g_learning_rate = 0.0001  # learning rate for the generator
d_learning_rate = 0.001   # learning rate for the discriminator
epochs          = 200
batch_size      = 64      # train batch size
eval_size       = 16      # evaluate size
smooth          = 0.1


In [24]:
## labels for the batch size and the test size
y_train_real, y_train_fake = make_labels(batch_size)
y_eval_real,  y_eval_fake  = make_labels(eval_size)

# create a GAN, a generator and a discriminator
gan, generator, discriminator = make_simple_GAN(
    sample_size, 
    g_hidden_size_1, 
    g_hidden_size_2,
    g_hidden_size_3,
    g_hidden_size_4, 
    d_hidden_size_1, 
    d_hidden_size_2,
    d_hidden_size_3,
    d_hidden_size_4,
    d_hidden_size_5,
    leaky_alpha, 
    g_learning_rate,
    d_learning_rate)


losses = []
losses1 = []
for e in range(epochs):
    for i in range(len(X_train_real)//batch_size):
        # real MNIST digit images
        X_batch_real = X_train_real[i*batch_size:(i+1)*batch_size]
        
        # latent samples and the generated digit images
        latent_samples = make_latent_samples(batch_size, sample_size)
        X_batch_fake = generator.predict_on_batch(latent_samples)
        
        # train the discriminator to detect real and fake images
        make_trainable(discriminator, True)
        discriminator.train_on_batch(X_batch_real, y_train_real * (1 - smooth))
        discriminator.train_on_batch(X_batch_fake, y_train_fake)

        # train the generator via GAN
        make_trainable(discriminator, False)
        gan.train_on_batch(latent_samples, y_train_real)
   # evaluate
    X_eval_real = X_test_real[np.random.choice(len(X_test_real), eval_size, replace=False)]
    
    latent_samples = make_latent_samples(eval_size, sample_size)
    X_eval_fake = generator.predict_on_batch(latent_samples)

    d_loss  = discriminator.test_on_batch(X_eval_real, y_eval_real)
    d_loss += discriminator.test_on_batch(X_eval_fake, y_eval_fake)
    g_loss  = gan.test_on_batch(latent_samples, y_eval_real) # we want the fake to be realistic!
    
    losses.append((d_loss, g_loss))

    #d_acc_real = discriminator.train_on_batch(X_batch_real, y_train_real * (1 - smooth))
    #d_acc_fake = discriminator.train_on_batch(X_batch_fake, y_train_fake)
    #d_acc  = 0.5 * (d_acc_real + d_acc_fake)

    #losses1.append(( d_acc))
    print("Epoch: {:>3}/{} Discriminator Loss: {:>6.4f} Generator Loss: {:>6.4f} ".format(
        e+1, epochs, d_loss, g_loss)) #Discriminator acc: {:>6.4f}:,d_acc

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch:   1/200 Discriminator Loss: 0.8995 Generator Loss: 0.7917 
Epoch:   2/200 Discriminator Loss: 0.5109 Generator Loss: 1.2248 
Epoch:   3/200 Discriminator Loss: 0.2794 Generator Loss: 1.9950 
Epoch:   4/200 Discriminator Loss: 0.3593 Generator Loss: 2.0901 
Epoch:   5/200 Discriminator Loss: 0.2499 Generator Loss: 2.1286 
Epoch:   6/200 Discriminator Loss: 0.3260 Generator Loss: 2.1271 
Epoch:   7/200 Discriminator Loss: 0.3235 Generator Loss: 2.5144 
Epoch:   8/200 Discriminator Loss: 0.2409 Generator Loss: 2.6873 
Epoch:   9/200 Discriminator Loss: 0.2045 Generator Loss: 2.6904 
Epoch:  10/200 Discriminator Loss: 0.1507 Generator Loss: 3.5253 
Epoch:  11/200 Discriminator Loss: 0.1559 Generator Loss: 3.8472 
Epoch:  12/200 Discriminator Loss: 0.1945 Generator Loss: 3.7882 
Epoch:  13/200 Discriminator Loss: 0.7057 Generator Loss: 4.0555 
Epoch:  14/200 Discriminator Loss: 0.2049 Generator Loss: 4.4470 
Epoch:  15/200 Discriminator Loss: 0.1441 Generator Loss: 4.4269 
Epoch:  16

In [25]:
losses = np.array(losses)

fig, ax = plt.subplots()
plt.plot(losses.T[0], label='Discriminator')
plt.plot(losses.T[1], label='Generator')
plt.title("Training Losses")
plt.legend()
plt.show()

In [26]:
rr = discriminator.predict_on_batch(X_test_real)

In [27]:
import keras
#import keras_metrics

r = tf.keras.metrics.Recall()#thresholds=rr.min())
r.update_state(y_test,rr )
r.result().numpy()


0.989899

In [28]:
p=tf.keras.metrics.Precision()#thresholds=rr.min())
p.update_state(y_test,rr )
p.result().numpy()

0.989899